In [ ]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
%store -r c3x
df = c3x
df

In [ ]:
df['Date']

In [ ]:
df.index = df.pop('Date')
df = df.sort_values('Date')
df.head()

In [ ]:
#Visualize the data
plt.figure(figsize = (16,8))
plt.title('Cash Balance')
plt.plot(df['Cash Balance'])
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Cash Balance', fontsize = 18)
plt.show()

In [ ]:
#Create a new dataframe
data = df.filter(['Cash Balance'])
#Convert to np.array
dataset = data.values
#Get the number of rows to train the model
training_data_len = math.ceil(len(dataset)*.8)

training_data_len

In [ ]:
#Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(dataset)

In [ ]:
#Make training datasets
train_data = scaled_data[0:training_data_len, :]
#Split the data into x_train and y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i <= 61:
        print(x_train)
        print(y_train)
        print()

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train.shape, y_train.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
#Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25, activation='swish'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss = 'mse', metrics = [RootMeanSquaredError()])

In [ ]:
#Train the model with validation split for validation data
history = model.fit(x_train, y_train, batch_size = 100, epochs = 10, validation_split=0.2, shuffle = True)

In [ ]:
#Create testing dataset
#Create a new array
test_data = scaled_data[training_data_len - 60: , :]
#Create datasets x_test, y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
predictions.shape

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#Visualise
plt.figure(figsize = (16,8))
plt.title('Cash Balance Univariate LSTM Model')
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Cash Balance', fontsize = 18)
plt.plot(train['Cash Balance'])
plt.plot(valid[['Cash Balance', 'Predictions']])
plt.legend(['Training', 'True Future', 'Predicted Future'], loc = 'lower left')
plt.show()

In [ ]:
#Visualise loss
#Visualizing loss
plt.figure(figsize = (16,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Training Loss', 'Validation Loss'], loc='lower left')
plt.show()

In [ ]:
plt.figure(figsize = (16,8))
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('Model RMSE')
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.legend(['Training RMSE', 'Validation RMSE'], loc = 'upper right')
plt.show()